<a href="https://colab.research.google.com/github/chloebs4590/Metis-Engineering/blob/main/train_stations_geocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Description

This notebook is for geocoding train station locations

In [ ]:
reset -fs

In [ ]:
!pip install googlemaps

In [ ]:
import pandas as pd
import os
import pickle

In [ ]:
# mount Google Drive
from google.colab import drive # import drive from google colab
from os.path import join
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
os.getcwd()

'/content'

In [ ]:
os.chdir('/content/drive/MyDrive/Data Science Metis/Engineering')

Read in data

In [ ]:
worksheet = gc.open('Amtrak Train and Bus Locations').sheet1
rows = worksheet.get_all_values()
stations_locations = pd.DataFrame.from_records(rows[1:])

In [ ]:
stations_locations.columns = ['x','y','objectid_1','objectid','station_descripton','bus_or_train',
                              'zip_code','state','city','address_2','address_1','name','code','station_name']

In [ ]:
stations_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   x                   1088 non-null   object
 1   y                   1088 non-null   object
 2   objectid_1          1088 non-null   object
 3   objectid            1088 non-null   object
 4   station_descripton  1088 non-null   object
 5   bus_or_train        1088 non-null   object
 6   zip_code            1088 non-null   object
 7   state               1088 non-null   object
 8   city                1088 non-null   object
 9   address_2           1088 non-null   object
 10  address_1           1088 non-null   object
 11  name                1088 non-null   object
 12  code                1088 non-null   object
 13  station_name        1088 non-null   object
dtypes: object(14)
memory usage: 119.1+ KB


In [ ]:
stations_locations.head()

,x,y,objectid_1,objectid,station_descripton,bus_or_train,zip_code,state,city,address_2,address_1,name,code,station_name
0,-9422620.154,5371788.236,1,1,Station Building (with waiting room),BUS,48801,MI,Alma,Transportation Center,1105 Willow Run Drive,,AAM,"Alma, MI"
1,-8216400.947,5273162.596,2,2,Curbside Bus Stop only (no shelter),BUS,12211,NY,Albany,,737 Albany Shaker Road,Albany Intl Airport,ABA,"Albany, NY"
2,-10053783.51,5610281.91,3,3,Curbside Bus Stop only (no shelter),BUS,54421,WI,Colby,,1210 North Division St.,,ABB,"Abbotsford-Colby, WI"
3,-8478455.55,4794767.297,4,4,Station Building (with waiting room),TRAIN,21001,MD,Aberdeen,,18 East Bel Air Avenue,,ABE,"Aberdeen, MD"
4,-8208827.1,5257492.746,14,16,Station Building (with waiting room),TRAIN,12144,NY,Rensselaer,,525 East Street,,ALB,"Albany-Rensselaer, NY"


Prepare df for geocoding addresses

In [ ]:
# drop bus stations

stations_locations = stations_locations[stations_locations['bus_or_train'] == 'TRAIN'].reset_index(drop=True)

In [ ]:
def state(x):
  if x == 'CA':
    return 'California'
  elif x == 'IL':
    return 'Illinois'
  elif x == 'NY':
    return 'New York'
  elif x == 'PA':
    return 'Pennsylvania'
  elif x == 'MI':
    return 'Michigan'
  elif x == 'VA':
    return 'Virginia'
  elif x == 'FL':
    return 'Florida'
  elif x == 'WA':
    return 'Washington'
  elif x == 'TX':
    return 'Texas'
  elif x == 'NC':
    return 'North Carolina'
  elif x == 'MO':
    return 'Missouri'
  elif x == 'CT':
    return 'Connecticut'
  elif x == 'MA':
    return 'Massachusetts'
  elif x == 'MT':
    return 'Montana'
  elif x == 'VT':
    return 'Vermont'
  elif x == 'IN':
    return 'Indiana'
  elif x == 'MS':
    return 'Mississippi'
  elif x == 'SC':
    return 'South Carolina'
  elif x == 'CO':
    return 'Colorado'
  elif x == 'WV':
    return 'West Virginia'
  elif x == 'AZ':
    return 'Arizona'
  elif x == 'OR':
    return 'Oregon'
  elif x == 'WI':
    return 'Wisconsin'
  elif x == 'ND':
    return 'North Dakota'
  elif x == 'LA':
    return 'Louisiana'
  elif x == 'OH':
    return 'Ohio'
  elif x == 'NM':
    return 'New Mexico'
  elif x == 'MN':
    return 'Minnesota'
  elif x == 'NJ':
    return 'New Jersey'
  elif x == 'KS':
    return 'Kansas'
  elif x == 'MD':
    return 'Maryland'
  elif x == 'ME':
    return 'Maine'
  elif x == 'AR':
    return 'Arkansas'
  elif x == 'ON':
    return 'Ontario'
  elif x == 'IA':
    return 'Iowa'
  elif x == 'GA':
    return 'Georgia'
  elif x == 'OK':
    return 'Oklahoma'
  elif x == 'NE':
    return 'Nebraska'
  elif x == 'NH':
    return 'New Hampshire'
  elif x == 'KY':
    return 'Kentucky'
  elif x == 'UT':
    return 'Utah'
  elif x == 'RI':
    return 'Rhode Island'
  elif x == 'AL':
    return 'Alabama'
  elif x == 'NV':
    return 'Nevada'
  elif x == 'TN':
    return 'Tennessee'
  elif x == 'QC':
    return 'Quebec'
  elif x == 'DE':
    return 'Delaware'
  elif x == 'ID':
    return 'Idaho'
  elif x == 'BC':
    return 'British Columbia'
  else:
    return 'DC'

In [ ]:
stations_locations['state_rev'] = stations_locations['state'].map(state)

In [ ]:
# create full address column for geocoding

stations_locations['full_address'] = stations_locations['address_1'] + "," + " "+ stations_locations['city'] + "," + " " + stations_locations['state_rev'] + " " + stations_locations['zip_code']

In [ ]:
stations_locations.head()

Geocode addresses (the below cells are commented out so they don't inadvertently run again)

In [ ]:
# import googlemaps
# #Set Google MAPS API_Key
# g_API = "my_key"
# gmaps_key = googlemaps.Client(key=g_API)

In [ ]:
# # Create columns to store longitude and latitude
# stations_locations["longitude"] = None
# stations_locations["latitude"] = None
# for i in stations_locations.index:
#     geocode_obj = gmaps_key.geocode(stations_locations.loc[i, "full_address"])
#     try:
#         lat = geocode_obj[0]['geometry']['location']['lat']
#         lon = geocode_obj[0]['geometry']['location']['lng']
#         stations_locations.loc[i,'latitude'] = lat
#         stations_locations.loc[i,'longitude'] = lon
#     except:
#         lat = None
#         lon = None

In [ ]:
# # pickle stations_locations df with geocoded longitude and latitude

# with open('stations_locations_geocoded.pkl', 'wb') as fid:
#      pickle.dump(stations_locations, fid)